In [2]:
import numpy as np

In [2]:
z = np.array([1, 2, 4])

arr = []
for i in range(len(z)):
    elem = np.exp(z[i] - 4) / 7
    arr.append(elem)
arr

[0.007112438338266277, 0.019333611890944673, 0.14285714285714285]

In [7]:
max = np.max(z)
np.exp(z - max) / np.sum(z - max)

array([0.00711244, 0.01933361, 0.14285714])

In [6]:
np.sum(z)

7

In [8]:
2 ** z

array([ 2,  4, 16], dtype=int32)

In [9]:
z-4

array([-3, -2,  0])

In [10]:
np.sum(z-4)

-5

In [32]:
Z = np.array([[1, 2, 3],
             [9, 4, 6],
             [7, 2, 9]])

In [33]:
Z

array([[1, 2, 3],
       [9, 4, 6],
       [7, 2, 9]])

In [34]:
np.max(Z, axis=1, keepdims=True)

array([[3],
       [9],
       [9]])

In [35]:
Z - np.max(Z, axis=1, keepdims=True)

array([[-2, -1,  0],
       [ 0, -5, -3],
       [-2, -7,  0]])

In [36]:
np.exp(Z - np.max(Z, axis=1, keepdims=True))

array([[1.35335283e-01, 3.67879441e-01, 1.00000000e+00],
       [1.00000000e+00, 6.73794700e-03, 4.97870684e-02],
       [1.35335283e-01, 9.11881966e-04, 1.00000000e+00]])

In [42]:
def forward(Z: np.ndarray) -> np.ndarray:
    Z_center = Z - np.max(Z, axis=1, keepdims=True)
        
    num = np.exp(Z_center)
    denom = np.sum(num, axis=1, keepdims=True)

    softmax = num / denom

    return softmax

def backward(Z: np.ndarray, dY: np.ndarray) -> np.ndarray:
    arr = []  
    s = forward(Z)
    ones = np.ones(Z.shape[0]).reshape(Z.shape[0], 1)
    s_minus = ones - s
    diags = s @ s_minus.T
    for row in diags:
        arr.append(np.diagflat(row))
    



    return np.array(arr)


In [54]:
Z = np.array([[1, 2, 3],
             [9, 4, 6],
             [7, 2, 9],
             [1, 5, 5]])

In [55]:
arr = []  
    s = forward(Z)
    ones = np.ones(Z.shape[0]).reshape(Z.shape[0], 1)
    s_minus = ones - s
    diags = s @ s_minus.T
    for row in diags:
        arr.append(np.diagflat(row))

array([[9.00305732e-02, 2.44728471e-01, 6.65240956e-01],
       [9.46499123e-01, 6.37746092e-03, 4.71234165e-02],
       [1.19107257e-01, 8.02538386e-04, 8.80090205e-01],
       [9.07471484e-03, 4.95462643e-01, 4.95462643e-01]])

In [58]:
A = np.ones(4).reshape(4,1) - s

In [59]:
diags = s @ A.T

array([[0.48945694, 0.88187697, 0.40360825, 0.54832714],
       [0.88187697, 0.10187812, 0.84578711, 0.9649031 ],
       [0.40360825, 0.84578711, 0.21125405, 0.56246969],
       [0.54832714, 0.9649031 , 0.56246969, 0.50895119]])

In [10]:
def im2col(X, kernel_shape, stride, pad):
    # n_channels = in_channels
    n_channels = X.shape[3]
    # sliding kernel shape
    kernel_height, kernel_width = kernel_shape
    # padding
    # X_pad is padded input
    # p
    X_pad, p = pad2d(X, pad, kernel_shape, stride)
    X_pad = X_pad.transpose(0, 3, 1, 2)
    X = X.transpose(0, 3, 1, 2)

    i, j, k = im2col_indices(X, kernel_shape, stride, p)
    X_col = X_pad[:, k, i, j]

    X_col = X_col.transpose(1, 2, 0).reshape(
        kernel_height * kernel_width * n_channels, -1
    )

    return X_col, p

In [12]:
import scipy.signal

In [14]:
Z = np.array([[[[1, 2, 3],
              [9, 4, 6],
              [7, 2, 9]]]])

In [19]:
Zsingle = np.array([[1, 2, 3],
              [9, 4, 6],
              [7, 2, 9]])

In [15]:
A = np.array([[1, 2],
              [2, 4]])

In [20]:
scipy.signal.convolve2d(Zsingle, A)

array([[ 1,  4,  7,  6],
       [11, 30, 28, 24],
       [25, 60, 41, 42],
       [14, 32, 26, 36]])

In [22]:
def pad2d(X, pad, kernel_shape, stride=None):
    p = pad
    if isinstance(p, int):
        p = (p, p, p, p)

    if isinstance(p, tuple):
        if len(p) == 2:
            p = (p[0], p[0], p[1], p[1])

        X_pad = np.pad(
            X,
            pad_width=((0, 0), (p[0], p[1]), (p[2], p[3]), (0, 0)),
            mode="constant",
            constant_values=0,
        )
    if p == "same" and kernel_shape and stride is not None:
        p = compute_pad(X, kernel_shape, stride)
        X_pad, p = pad2d(X, p, kernel_shape, stride)
    return X_pad, p


In [24]:
def im2col_indices(X, kernel_shape, stride, pad):
    p = pad

    n_examples, in_channels, in_rows, in_cols = X.shape
    kernel_height, kernel_width = kernel_shape

    out_rows = (in_rows + p[0] + p[1] - kernel_height) // stride + 1
    out_cols = (in_cols + p[2] + p[3] - kernel_width) // stride + 1

    i0 = np.repeat(np.arange(kernel_height), kernel_width)
    i0 = np.tile(i0, in_channels)
    i1 = stride * np.repeat(np.arange(out_rows), out_cols)
    j0 = np.tile(np.arange(kernel_width), kernel_height * in_channels)
    j1 = stride * np.tile(np.arange(out_cols), out_rows)

    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    k = np.repeat(np.arange(in_channels), kernel_height * kernel_width).reshape(
        -1, 1
    )

    return i, j, k

In [25]:
im2col(Z, (2, 2), 1, 0)

(array([], shape=(12, 0), dtype=int64), (0, 0, 0, 0))

In [31]:
W = np.ones(420).reshape(5,4, 7, 3)

In [33]:
print(W.shape)
print(W.T.shape)

(5, 4, 7, 3)
(3, 7, 4, 5)


In [34]:
W.shape

(5, 4, 7, 3)

In [36]:
W.T.shape

(3, 7, 4, 5)

In [38]:
np.transpose(W, [3, 2, 0, 1]).shape

(3, 7, 5, 4)

In [43]:
W.reshape((W.shape[0], -1)).shape

(5, 84)

In [49]:
W_T = np.transpose(W, [3, 2, 0, 1])

In [50]:
W_T.shape

(3, 7, 5, 4)

In [ ]:
W_T.reshape(3, )

In [ ]:
def forward(self, X):
        """
            Performs a forward convolution.
           
            Parameters:
            - X : Last conv layer of shape (m, n_C_prev, n_H_prev, n_W_prev).
            Returns:
            - out: previous layer convolved.
        """
        # m -> number of examples.
        # n_C_prev -> number of channels
        # n_H_prev -> height of the previous layer
        # n_W_prev -> width of the previous layer
        m, n_C_prev, n_H_prev, n_W_prev = X.shape

        # self.f -> filter size
        # self.n_F -> number of filters, out_channels??
        # self.s -> stride
        n_C = self.n_F
        n_H = int((n_H_prev + 2 * self.p - self.f)/ self.s) + 1
        n_W = int((n_W_prev + 2 * self.p - self.f)/ self.s) + 1
        
        X_col = im2col(X, self.f, self.f, self.s, self.p)
        w_col = self.W['val'].reshape((self.n_F, -1))
        b_col = self.b['val'].reshape(-1, 1)
        # Perform matrix multiplication.
        out = w_col @ X_col + b_col
        # Reshape back matrix to image.
        out = np.array(np.hsplit(out, m)).reshape((m, n_C, n_H, n_W))
        self.cache = X, X_col, w_col
        return out

In [67]:
# kernel height = 3
# kernel width = 3
# in_channels = 3
# out_channels = 2

# current --> height, width, channel, filters
# result --> split by filter, then channel, then height, then width
A = np.random.randint(0, 10, (3, 3, 3, 2))

In [70]:
A

array([[[[3, 1],
         [1, 8],
         [6, 1]],

        [[2, 9],
         [6, 3],
         [5, 8]],

        [[9, 2],
         [5, 3],
         [5, 5]]],


       [[[3, 5],
         [1, 8],
         [5, 6]],

        [[5, 4],
         [3, 6],
         [7, 1]],

        [[3, 2],
         [5, 4],
         [3, 7]]],


       [[[0, 1],
         [2, 8],
         [6, 4]],

        [[8, 3],
         [0, 2],
         [2, 1]],

        [[4, 2],
         [1, 9],
         [8, 2]]]])

In [64]:
np.transpose(A, (3, 2, 0, 1))

array([[[[4, 6, 9],
         [8, 4, 2]],

        [[3, 3, 4],
         [3, 2, 5]],

        [[2, 4, 2],
         [5, 9, 1]]],


       [[[2, 5, 6],
         [1, 8, 9]],

        [[5, 7, 0],
         [9, 8, 4]],

        [[4, 2, 0],
         [8, 9, 2]]],


       [[[7, 4, 6],
         [7, 1, 7]],

        [[2, 1, 5],
         [2, 6, 7]],

        [[0, 7, 9],
         [0, 4, 3]]]])

In [133]:
A.reshape(27, 2, order="F")

array([[3, 1],
       [3, 5],
       [0, 1],
       [2, 9],
       [5, 4],
       [8, 3],
       [9, 2],
       [3, 2],
       [4, 2],
       [1, 8],
       [1, 8],
       [2, 8],
       [6, 3],
       [3, 6],
       [0, 2],
       [5, 3],
       [5, 4],
       [1, 9],
       [6, 1],
       [5, 6],
       [6, 4],
       [5, 8],
       [7, 1],
       [2, 1],
       [5, 5],
       [3, 7],
       [8, 2]])

In [72]:
# result --> split by filter, then channel, then height, then width
A_T = A.transpose(3, 2, 0, 1)
A_T

array([[[[3, 2, 9],
         [3, 5, 3],
         [0, 8, 4]],

        [[1, 6, 5],
         [1, 3, 5],
         [2, 0, 1]],

        [[6, 5, 5],
         [5, 7, 3],
         [6, 2, 8]]],


       [[[1, 9, 2],
         [5, 4, 2],
         [1, 3, 2]],

        [[8, 3, 3],
         [8, 6, 4],
         [8, 2, 9]],

        [[1, 8, 5],
         [6, 1, 7],
         [4, 1, 2]]]])

In [107]:
arr = []
for i in range(A_T.shape[0]):
    arr.append(A_T[i].flatten())
arr_T = np.array(arr).T

In [108]:
A_T

array([[[[3, 2, 9],
         [3, 5, 3],
         [0, 8, 4]],

        [[1, 6, 5],
         [1, 3, 5],
         [2, 0, 1]],

        [[6, 5, 5],
         [5, 7, 3],
         [6, 2, 8]]],


       [[[1, 9, 2],
         [5, 4, 2],
         [1, 3, 2]],

        [[8, 3, 3],
         [8, 6, 4],
         [8, 2, 9]],

        [[1, 8, 5],
         [6, 1, 7],
         [4, 1, 2]]]])

In [127]:
print(A_T.T.shape)
print(arr_T.shape)

(3, 3, 3, 2)
(27, 2)


In [128]:
A_T.T

array([[[[3, 1],
         [1, 8],
         [6, 1]],

        [[3, 5],
         [1, 8],
         [5, 6]],

        [[0, 1],
         [2, 8],
         [6, 4]]],


       [[[2, 9],
         [6, 3],
         [5, 8]],

        [[5, 4],
         [3, 6],
         [7, 1]],

        [[8, 3],
         [0, 2],
         [2, 1]]],


       [[[9, 2],
         [5, 3],
         [5, 5]],

        [[3, 2],
         [5, 4],
         [3, 7]],

        [[4, 2],
         [1, 9],
         [8, 2]]]])

In [131]:
arr_T.reshape(3, 3, 3, 2, order='F')

array([[[[3, 1],
         [1, 8],
         [6, 1]],

        [[3, 5],
         [1, 8],
         [5, 6]],

        [[0, 1],
         [2, 8],
         [6, 4]]],


       [[[2, 9],
         [6, 3],
         [5, 8]],

        [[5, 4],
         [3, 6],
         [7, 1]],

        [[8, 3],
         [0, 2],
         [2, 1]]],


       [[[9, 2],
         [5, 3],
         [5, 5]],

        [[3, 2],
         [5, 4],
         [3, 7]],

        [[4, 2],
         [1, 9],
         [8, 2]]]])

In [132]:
arr_T.reshape(3, 3, 3, 2, order='F') == A_T.T

array([[[[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]]]])

In [130]:
arr_T

array([[3, 1],
       [2, 9],
       [9, 2],
       [3, 5],
       [5, 4],
       [3, 2],
       [0, 1],
       [8, 3],
       [4, 2],
       [1, 8],
       [6, 3],
       [5, 3],
       [1, 8],
       [3, 6],
       [5, 4],
       [2, 8],
       [0, 2],
       [1, 9],
       [6, 1],
       [5, 8],
       [5, 5],
       [5, 6],
       [7, 1],
       [3, 7],
       [6, 4],
       [2, 1],
       [8, 2]])

In [126]:
A_T.T

array([[[[3, 1],
         [1, 8],
         [6, 1]],

        [[3, 5],
         [1, 8],
         [5, 6]],

        [[0, 1],
         [2, 8],
         [6, 4]]],


       [[[2, 9],
         [6, 3],
         [5, 8]],

        [[5, 4],
         [3, 6],
         [7, 1]],

        [[8, 3],
         [0, 2],
         [2, 1]]],


       [[[9, 2],
         [5, 3],
         [5, 5]],

        [[3, 2],
         [5, 4],
         [3, 7]],

        [[4, 2],
         [1, 9],
         [8, 2]]]])

In [124]:
arr_T.reshape(3, 3, 3, 2)

array([[[[3, 1],
         [2, 9],
         [9, 2]],

        [[3, 5],
         [5, 4],
         [3, 2]],

        [[0, 1],
         [8, 3],
         [4, 2]]],


       [[[1, 8],
         [6, 3],
         [5, 3]],

        [[1, 8],
         [3, 6],
         [5, 4]],

        [[2, 8],
         [0, 2],
         [1, 9]]],


       [[[6, 1],
         [5, 8],
         [5, 5]],

        [[5, 6],
         [7, 1],
         [3, 7]],

        [[6, 4],
         [2, 1],
         [8, 2]]]])

In [117]:
arr_T.T

array([[3, 2, 9, 3, 5, 3, 0, 8, 4, 1, 6, 5, 1, 3, 5, 2, 0, 1, 6, 5, 5, 5,
        7, 3, 6, 2, 8],
       [1, 9, 2, 5, 4, 2, 1, 3, 2, 8, 3, 3, 8, 6, 4, 8, 2, 9, 1, 8, 5, 6,
        1, 7, 4, 1, 2]])

In [119]:
arr_T.T.reshape(2, 3, 3, 3, order="C")

array([[[[3, 2, 9],
         [3, 5, 3],
         [0, 8, 4]],

        [[1, 6, 5],
         [1, 3, 5],
         [2, 0, 1]],

        [[6, 5, 5],
         [5, 7, 3],
         [6, 2, 8]]],


       [[[1, 9, 2],
         [5, 4, 2],
         [1, 3, 2]],

        [[8, 3, 3],
         [8, 6, 4],
         [8, 2, 9]],

        [[1, 8, 5],
         [6, 1, 7],
         [4, 1, 2]]]])

In [100]:
arr_T.shape

(27, 2)

In [101]:
arr_T.reshape(3, 3, 3, 2)

array([[[[3, 1],
         [2, 9],
         [9, 2]],

        [[3, 5],
         [5, 4],
         [3, 2]],

        [[0, 1],
         [8, 3],
         [4, 2]]],


       [[[1, 8],
         [6, 3],
         [5, 3]],

        [[1, 8],
         [3, 6],
         [5, 4]],

        [[2, 8],
         [0, 2],
         [1, 9]]],


       [[[6, 1],
         [5, 8],
         [5, 5]],

        [[5, 6],
         [7, 1],
         [3, 7]],

        [[6, 4],
         [2, 1],
         [8, 2]]]])

In [78]:
X = np.random.randint(0, 10, (27, 40))
X

array([[7, 5, 4, ..., 5, 4, 0],
       [1, 5, 6, ..., 8, 7, 4],
       [0, 9, 7, ..., 7, 6, 3],
       ...,
       [0, 0, 3, ..., 3, 6, 9],
       [3, 1, 1, ..., 5, 1, 9],
       [1, 2, 2, ..., 0, 2, 5]])

In [82]:
A_T.shape

(2, 3, 3, 3)

In [ ]:
# result --> split by filter, then channel, then height, then width
array([[[[3, 2, 9],
         [3, 5, 3],
         [0, 8, 4]],

        [[1, 6, 5],
         [1, 3, 5],
         [2, 0, 1]],

        [[6, 5, 5],
         [5, 7, 3],
         [6, 2, 8]]],


       [[[1, 9, 2],
         [5, 4, 2],
         [1, 3, 2]],

        [[8, 3, 3],
         [8, 6, 4],
         [8, 2, 9]],

        [[1, 8, 5],
         [6, 1, 7],
         [4, 1, 2]]]])

In [ ]:
# kernel height = 3
# kernel width = 3
# in_channels = 3
# out_channels = 2

array([[[[3, 1],
         [1, 8],
         [6, 1]],

        [[2, 9],
         [6, 3],
         [5, 8]],

        [[9, 2],
         [5, 3],
         [5, 5]]],


       [[[3, 5],
         [1, 8],
         [5, 6]],

        [[5, 4],
         [3, 6],
         [7, 1]],

        [[3, 2],
         [5, 4],
         [3, 7]]],


       [[[0, 1],
         [2, 8],
         [6, 4]],

        [[8, 3],
         [0, 2],
         [2, 1]],

        [[4, 2],
         [1, 9],
         [8, 2]]]])

In [135]:
A_T * A_T

array([[[[ 9,  4, 81],
         [ 9, 25,  9],
         [ 0, 64, 16]],

        [[ 1, 36, 25],
         [ 1,  9, 25],
         [ 4,  0,  1]],

        [[36, 25, 25],
         [25, 49,  9],
         [36,  4, 64]]],


       [[[ 1, 81,  4],
         [25, 16,  4],
         [ 1,  9,  4]],

        [[64,  9,  9],
         [64, 36, 16],
         [64,  4, 81]],

        [[ 1, 64, 25],
         [36,  1, 49],
         [16,  1,  4]]]])

In [138]:
A_T.sum()

223

In [139]:
W = np.random.randint(0, 10, (3, 3, 3))

In [140]:
W

array([[[8, 1, 9],
        [8, 1, 0],
        [9, 2, 9]],

       [[1, 6, 1],
        [8, 9, 0],
        [9, 3, 6]],

       [[4, 0, 6],
        [4, 7, 0],
        [6, 7, 7]]])

In [147]:
np.sum(W, axis=(0, 1, 2))

131